In [1]:
'''
This is an implementation of Speech to Text, a Large Language Model and a Text to Speech Generator.
It should take a few minutes to download the models. Once loaded inference on GPU should be around 10 seconds.

Notes;

1. Under Runtime and Change Runtime Type, ensure Runtime is set to T4.

2. Under Runtime, select Run all

3. In the Gradio box

    A) The slider controls the creativity vs precision of the responses.
    B) Click record to start and stop audio recording.
    C) Press submit to pass to the models.
    D) Press play on the right had side when the buttom appears.
    E) Enjoy

'''


'\nThis is an implementation of Speech to Text, a Large Language Model and a Text to Speech Generator.\nIt should take a few minutes to download the models. Once loaded inference on GPU should be around 10 seconds.\n\nNotes;\n\n1. Under Runtime and Change Runtime Type, ensure Runtime is set to T4.\n\n2. Under Runtime, select Run all\n\n3. In the Gradio box\n\n    A) The slider controls the creativity vs precision of the responses.\n    B) Click record to start and stop audio recording.\n    C) Press submit to pass to the models.\n    D) Press play on the right had side when the buttom appears.\n    E) Enjoy\n\n'

In [2]:
# Install libraries

!pip install SentencePiece transformers datasets gradio==3.47.1 accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.

In [3]:
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import soundfile as sf
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import gradio as gr
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
int_device = 0 if device == "cuda:0" else -1


# Load Speech to Text model

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en", device = int_device)


# Load Large Language Model model

tokenizer_lmm = AutoTokenizer.from_pretrained("smallcloudai/Refact-1_6B-fim", trust_remote_code=True)
model_mm = AutoModelForCausalLM.from_pretrained("smallcloudai/Refact-1_6B-fim", trust_remote_code=True).to(device)

prompt_template = "<empty_output>SYSTEM {system}\n" \
            "<empty_output>USER {query}\n" \
            "<empty_output>ASSISTANT"


# Load Text to Speech Generator model

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device = int_device)

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[6800]["xvector"]).unsqueeze(0).to(device)

print(f'Running with {device} -- {int_device}')


config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Running with cpu -- -1


In [4]:
from scipy.signal import resample
import torchaudio
import soundfile as sf

# Speech to Text Funciton

def gen_stt(audio_input):

    if type(audio_input) == str:

      audio_array, sr = sf.read('/content/sample_audio.wav')

    else:

      sr, audio_array = audio_input
      audio_array = audio_array.astype(np.float32)
      audio_array /= np.max(np.abs(audio_array))

    print('\nActivating STT')
    start_time = time.time()

    transcription = transcriber({"sampling_rate": sr, "raw": audio_array})["text"]

    stop_time = time.time()
    print(f'SST Done, Time {stop_time - start_time}')

    print(transcription)

    return transcription

# Convert numbers to their alpha representation

import re
import inflect

def convert_numbers_to_words(text):
    # Initialize inflect engine
    p = inflect.engine()

    # Regular expression to find numbers
    regex = r'\d+'
    matches = re.findall(regex, text)

    # Replace numbers with their textual representations
    for match in matches:
        word_representation = p.number_to_words(int(match))
        text = text.replace(match, word_representation)

    new_string = text.replace('-', ' ')

    return new_string


# Large Language Model Funciton

def gen_llm(text_inp, temp):

    print('\nActivating LLM')

    start_time = time.time()

    prompt = prompt_template.format(system="You are an AI assistant, keep your responses less than 30 words.",
                                    query = text_inp)

    inputs = tokenizer_lmm.encode(prompt, return_tensors="pt").to(device)
    outputs = model_mm.generate(inputs, max_length=150, temperature= temp)
    ten_gen_outs = tokenizer_lmm.decode(outputs[0], skip_special_tokens = True)

    assistant_response = ten_gen_outs[ten_gen_outs.rfind("ASSISTANT") + len("ASSISTANT"):].strip()

    converted_assistant_response = convert_numbers_to_words(assistant_response)

    stop_time = time.time()

    print(f'LLM Done, Time {stop_time - start_time}')
    print(converted_assistant_response)

    return converted_assistant_response


# Text to Speech Generator Funciton

def gen_tts(text_input):

    print('\nActivating TTS')
    start_time = time.time()

    speech = synthesiser(text_input, forward_params={"speaker_embeddings": speaker_embeddings})
    scaled_array = (speech["audio"] * np.iinfo(np.int16).max).astype(np.int16)

    stop_time = time.time()
    print(stop_time - start_time)
    print(f'TTS Done, Time {stop_time - start_time}')

    # return path
    return 16000, scaled_array


# # Converse Funciton

def converse(audio_input, temp):

    text_input = gen_stt(audio_input)

    response = gen_llm(text_input, temp)

    speech_array = gen_tts(response)

    return speech_array

In [5]:
#
# Launch Gradio
#

In [6]:
demo = gr.Interface(
    converse,
    [gr.Audio(source="microphone"),
    gr.Slider(0.01, 1, value=0.5, label="Precision vs Creativity", info="Choose between 0 and 1"),
    ],
    outputs = "audio",
)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8a2889778354d40d80.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8a2889778354d40d80.gradio.live
